In [1]:
# general
import warnings, gc, sys, os, operator
warnings.filterwarnings('ignore')

# data
import pandas as pd
import numpy as np
from datetime import datetime as dt
import calendar

In [10]:
read_path = '../dataset/origin/'
write_path = '../dataset/fin/'
hist = pd.read_hdf(write_path+'hist.h5')
new = pd.read_hdf(write_path+'new.h5')
mer = pd.read_hdf(write_path+'mer.h5')

In [4]:
trans = hist.append(new)
del hist, new
trans = trans.reset_index()
trans = trans.drop(columns='index')
gc.collect()

90

In [ ]:
trans = transPrep(trans)
trans.to_hdf(write_path+'trans.h5', 'trans')

trans = transFE_basic(trans)
trans.to_hdf(write_path+'trans.h5', 'trans')

purchase_amount_by_ym_trans = purchase_amount_by_ym(trans)
purchase_amount_by_ym_trans = renameCols(purchase_amount_by_ym_trans)
purchase_amount_by_ym_trans.to_csv(write_path+'purchase_amount_by_ym_trans.csv', index=False)

transaction_count_by_ym_trans = transaction_count_by_ym(trans)
transaction_count_by_ym_trans = renameCols(transaction_count_by_ym_trans)
transaction_count_by_ym_trans.to_csv(write_path+'transaction_count_by_ym_trans.csv', index=False)
del transaction_count_by_ym_trans

purchase_amount_by_ym_merchant_trans = purchase_amount_by_ym_merchant(trans)
purchase_amount_by_ym_merchant_trans = renameCols(purchase_amount_by_ym_merchant_trans)
purchase_amount_by_ym_merchant_trans.to_csv(write_path+'purchase_amount_by_ym_merchant_trans.csv', index=False)
del purchase_amount_by_ym_merchant_trans

trans_count_by_ym_merchant_trans = trans_count_by_ym_merchant(trans)
trans_count_by_ym_merchant_trans = renameCols(trans_count_by_ym_merchant_trans)
trans_count_by_ym_merchant_trans.to_csv(write_path+'trans_count_by_ym_merchant_trans.csv', index=False)
del trans_count_by_ym_merchant_trans

purchase_amount_by_ym_city_trans = purchase_amount_by_ym_city(trans)
purchase_amount_by_ym_city_trans = renameCols(purchase_amount_by_ym_city_trans)
purchase_amount_by_ym_city_trans.to_csv(write_path+'purchase_amount_by_ym_city_trans.csv', index=False)
del purchase_amount_by_ym_city_trans

trans_count_by_ym_city_trans = trans_count_by_ym_city(trans)
trans_count_by_ym_city_trans = renameCols(trans_count_by_ym_city_trans)
trans_count_by_ym_city_trans.to_csv(write_path+'trans_count_by_ym_city_trans.csv', index=False)
del trans_count_by_ym_city_trans

transFE_trans = transFE(trans)
transFE_trans2 = transAgg(transFE_trans)
transFE_trans2 = renameCols(transFE_trans2)
transFE_trans2.to_csv(write_path+'fe01_trans.csv', index=False)
del transFE_trans2, transFE_trans

fe2_trans = transFE2(trans)
fe2_trans = renameCols(fe2_trans)
fe2_trans.to_csv(write_path+'fe2_trans.csv', index=False)
del fe2_trans

mervisit_trans = mervisit(trans)
mervisit_trans = renameCols(mervisit_trans)
mervisit_trans.to_csv(write_path+'mervisit_trans.csv', index=False)
del mervisit_trans

mertry_trans = mertry(trans)
mertry_trans = renameCols(mertry_trans)
mertry_trans.to_csv(write_path+'mertry_trans.csv', index=False)
del mertry_trans

trans = transYM(trans)
tAgg = transYMAgg(trans)
tAgg = renameCols(tAgg)
tAgg.to_csv(write_path+'modeKey_trans.csv', index=False)
del tAgg

merIdCnvrt_trans = merIdCnvrt(trans)
merIdCnvrt_trans = renameCols(merIdCnvrt_trans)
merIdCnvrt_trans.to_csv(write_path+'merIdCnvrt_trans.csv', index=False)
del merIdCnvrt_trans

In [10]:
merCols = mer.columns.tolist()
for col in ['category_1', 'city_id', 'merchant_category_id', 'state_id', 'subsector_id', 'numerical_1', 'numerical_2', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']:
    merCols.remove(col)
cols = ['card_id', 'merchant_id']
transMer = trans[cols].merge(mer[merCols], how='left', on='merchant_id')

mer_trans = merAgg(transMer)
mer_trans = renameCols(mer_trans)
mer_trans.to_csv(write_path+'mer_trans.csv', index=False)
del mer_trans

In [6]:
def renameCols(df, prefix=None, postfix=None):
    rename = []
    for col in df.columns.tolist():
        if type(col) != str :
            if col[1]== '' : rename.append(col[0])
            else : rename.append(col[0] + "_" + col[1])
        else : rename.append(col)
    df.columns = pd.Index(rename)
    if prefix is not None :
        df.columns = [prefix+ c if c!='card_id' else c for c in df.columns]
    if postfix is not None :
        df.columns = [c+postfix if c!='card_id' else c for c in df.columns]
    return df

def null_cnt(x):
    return np.sum(x.isna() * 1)

def over_550(data):
    return len(data[data > 550])

from scipy import stats
def mode(x):
    return stats.mode(x)[0][0]

def quantileRange(x):
    q1 = np.quantile(x, 0.25)
    q3 = np.quantile(x, 0.75)
    return q3 - q1

from collections import Counter
def modeKey(x):
    return Counter(x).most_common(1)[0][0]

def top3_val_mean(x):
    cnt = Counter(x).most_common(3)
    ret = 0
    for key, val in cnt:
        ret+=val
    return ret/3

def top3_key_mean(x):
    cnt = Counter(x).most_common(3)
    ret = 0
    for key, val in cnt:
        ret+=key
    return ret/3

In [9]:
def cat2(x):
    if pd.isna(x): return np.nan
    else : return int(x-1) 

def transPrep(df):
    df['authorized_flag_new'] = df['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    df['category_1_new'] = df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    df['category_3_new'] = df['category_3'].map({'A':0, 'B':1, 'C':2})
    df['category_2_new'] = df['category_2'].apply(cat2)
    df['installments_new'] = df['installments'].apply(lambda x : np.nan if x==999 or x==-1 else x)
    df['purchase_amount_new'] = np.round(df['purchase_amount'] / 0.00150265118 + 497.06).astype(int)
    return df

def transFE_basic(df):
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['purchase_year'] = df['purchase_date'].dt.year
    df['purchase_month'] = df['purchase_date'].dt.month
    return df

In [1]:
def purchase_amount_by_ym(df):
    purchase_amount_sum_by_ym = df.groupby(['purchase_year', 'purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_ym_card = df.groupby(['card_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_ym_dict = {}
    for (card_id, year, month), val in zip(purchase_amount_sum_by_ym_card.index.tolist(), purchase_amount_sum_by_ym_card.values):
        if card_id not in purchase_amount_sum_by_ym_dict : purchase_amount_sum_by_ym_dict[card_id] = {}
        if year not in purchase_amount_sum_by_ym_dict[card_id]: purchase_amount_sum_by_ym_dict[card_id][year] = {}
        purchase_amount_sum_by_ym_dict[card_id][year][month] = val
    def purchase_amount_diff_by_ym(x):
        month = x['purchase_month']
        try :
            if month==1 : temp = purchase_amount_sum_by_ym_dict[x['card_id']][x['purchase_year']-1][12]
            else : temp = purchase_amount_sum_by_ym_dict[x['card_id']][x['purchase_year']][month-1]
        except KeyError : return np.nan
        return x['purchase_amount_sum_by_ym_card']-temp
    purchase_amount_sum_by_ym_card = purchase_amount_sum_by_ym_card.reset_index()
    purchase_amount_sum_by_ym_card.columns = ['card_id', 'purchase_year', 'purchase_month','purchase_amount_sum_by_ym_card']
    purchase_amount_sum_by_ym_card['purchase_amount_diff_by_ym_card'] = purchase_amount_sum_by_ym_card.apply(lambda x : purchase_amount_diff_by_ym(x), axis=1)
    purchase_amount_sum_by_ym_card['purchase_amount_sum_rate_as_ym_by_card'] = purchase_amount_sum_by_ym_card.apply(lambda x : x['purchase_amount_sum_by_ym_card']/purchase_amount_sum_by_ym[x['purchase_year']][x['purchase_month']]*100, axis=1)

    aggs = {
        'purchase_amount_sum_by_ym_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_rate_as_ym_by_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_diff_by_ym_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
    }
    return purchase_amount_sum_by_ym_card.groupby('card_id').agg(aggs).reset_index()

In [11]:
def transaction_count_by_ym(df):
    trans_count_by_ym = df.groupby(['purchase_year', 'purchase_month'])['purchase_amount_new'].agg('count')
    trans_count_by_ym_card = df.groupby(['card_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('count')
    trans_count_by_ym_dict = {}
    for (card_id, year, month), val in zip(trans_count_by_ym_card.index.tolist(), trans_count_by_ym_card.values):
        if card_id not in trans_count_by_ym_dict : trans_count_by_ym_dict[card_id] = {}
        if year not in trans_count_by_ym_dict[card_id]: trans_count_by_ym_dict[card_id][year] = {}
        trans_count_by_ym_dict[card_id][year][month] = val
    def trans_count_diff_by_ym(x):
        month = x['purchase_month']
        try :
            if month==1 : temp = trans_count_by_ym_dict[x['card_id']][x['purchase_year']-1][12]
            else : temp = trans_count_by_ym_dict[x['card_id']][x['purchase_year']][month-1]
        except KeyError : return np.nan
        return x['trans_count_by_ym_card']-temp
    trans_count_by_ym_card = trans_count_by_ym_card.reset_index()
    trans_count_by_ym_card.columns = ['card_id', 'purchase_year', 'purchase_month', 'trans_count_by_ym_card']
    trans_count_by_ym_card['trans_count_diff_by_ym_card'] = trans_count_by_ym_card.apply(lambda x : trans_count_diff_by_ym(x), axis=1)
    trans_count_by_ym_card['trans_count_rate_as_ym_by_card'] = trans_count_by_ym_card.apply(lambda x : x['trans_count_by_ym_card']/trans_count_by_ym[x['purchase_year']][x['purchase_month']]*100, axis=1)
    
    aggs = {
        'trans_count_by_ym_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_rate_as_ym_by_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_diff_by_ym_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
    }
    return trans_count_by_ym_card.groupby('card_id').agg(aggs)

In [12]:
def purchase_amount_by_ym_merchant(df):
    purchase_amount_sum_by_ym = df.groupby(['purchase_year', 'purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_merchant = df.groupby('merchant_id')['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_ym_merchant = df.groupby(['merchant_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_ym_merchant_card = df.groupby(['card_id', 'merchant_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_ym_merchant_dict = {}
    for (card_id, merchant, year, month), val in zip(purchase_amount_sum_by_ym_merchant_card.index.tolist(), purchase_amount_sum_by_ym_merchant_card.values):
        if card_id not in purchase_amount_sum_by_ym_merchant_dict : purchase_amount_sum_by_ym_merchant_dict[card_id] = {}
        if merchant not in purchase_amount_sum_by_ym_merchant_dict[card_id] : purchase_amount_sum_by_ym_merchant_dict[card_id][merchant] = {}
        if year not in purchase_amount_sum_by_ym_merchant_dict[card_id][merchant]: purchase_amount_sum_by_ym_merchant_dict[card_id][merchant][year] = {}
        purchase_amount_sum_by_ym_merchant_dict[card_id][merchant][year][month] = val
    def purchase_amount_diff_by_ym_merchant(x):
        month = x['purchase_month']
        try :
            if month==1 : temp = purchase_amount_sum_by_ym_merchant_dict[x['card_id']][x['merchant_id']][x['purchase_year']-1][12]
            else : temp = purchase_amount_sum_by_ym_merchant_dict[x['card_id']][x['merchant_id']][x['purchase_year']][month-1]
        except KeyError : return np.nan
        return x['purchase_amount_sum_by_ym_merchant_card']-temp
    purchase_amount_sum_by_ym_merchant_card = purchase_amount_sum_by_ym_merchant_card.reset_index()
    purchase_amount_sum_by_ym_merchant_card.columns = ['card_id', 'merchant_id', 'purchase_year', 'purchase_month','purchase_amount_sum_by_ym_merchant_card']
    purchase_amount_sum_by_ym_merchant_card['purchase_amount_diff_by_ym_merchant_card'] = purchase_amount_sum_by_ym_merchant_card.apply(lambda x : purchase_amount_diff_by_ym_merchant(x), axis=1)
    purchase_amount_sum_by_ym_merchant_card['purchase_amount_sum_by_ym_merchant_card_rate_as_ym'] = purchase_amount_sum_by_ym_merchant_card.apply(lambda x : x['purchase_amount_sum_by_ym_merchant_card']/purchase_amount_sum_by_ym[x['purchase_year']][x['purchase_month']]*100, axis=1)
    purchase_amount_sum_by_ym_merchant_card['purchase_amount_sum_by_ym_merchant_card_rate_as_merchant'] = purchase_amount_sum_by_ym_merchant_card.apply(lambda x : x['purchase_amount_sum_by_ym_merchant_card']/purchase_amount_sum_by_merchant[x['merchant_id']]*100, axis=1)
    purchase_amount_sum_by_ym_merchant_card['purchase_amount_sum_by_ym_merchant_card_rate_as_ym_merchant'] = purchase_amount_sum_by_ym_merchant_card.apply(lambda x : x['purchase_amount_sum_by_ym_merchant_card']/purchase_amount_sum_by_ym_merchant[x['merchant_id']][x['purchase_year']][x['purchase_month']]*100, axis=1)
    
    aggs = {
        'purchase_amount_sum_by_ym_merchant_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_ym_merchant_card_rate_as_ym':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_ym_merchant_card_rate_as_merchant' : ['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_ym_merchant_card_rate_as_ym_merchant' : ['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_diff_by_ym_merchant_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
    }
    return purchase_amount_sum_by_ym_merchant_card.groupby('card_id').agg(aggs)

In [13]:
def trans_count_by_ym_merchant(df):
    trans_count_by_ym = df.groupby(['purchase_year', 'purchase_month'])['purchase_amount_new'].agg('count')
    trans_count_by_merchant = df.groupby('merchant_id')['purchase_amount_new'].agg('count')
    trans_count_by_ym_merchant = df.groupby(['merchant_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('count')
    trans_count_by_ym_merchant_card = df.groupby(['card_id', 'merchant_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('count')
    trans_count_by_ym_merchant_dict = {}
    for (card_id, merchant, year, month), val in zip(trans_count_by_ym_merchant_card.index.tolist(), trans_count_by_ym_merchant_card.values):
        if card_id not in trans_count_by_ym_merchant_dict : trans_count_by_ym_merchant_dict[card_id] = {}
        if merchant not in trans_count_by_ym_merchant_dict[card_id] : trans_count_by_ym_merchant_dict[card_id][merchant] = {}
        if year not in trans_count_by_ym_merchant_dict[card_id][merchant]: trans_count_by_ym_merchant_dict[card_id][merchant][year] = {}
        trans_count_by_ym_merchant_dict[card_id][merchant][year][month] = val
    def trans_count_diff_by_ym_merchant(x):
        month = x['purchase_month']
        try :
            if month==1 : temp = trans_count_by_ym_merchant_dict[x['card_id']][x['merchant_id']][x['purchase_year']-1][12]
            else : temp = trans_count_by_ym_merchant_dict[x['card_id']][x['merchant_id']][x['purchase_year']][month-1]
        except KeyError : return np.nan
        return x['trans_count_by_ym_merchant_card']-temp
    trans_count_by_ym_merchant_card = trans_count_by_ym_merchant_card.reset_index()
    trans_count_by_ym_merchant_card.columns = ['card_id', 'merchant_id', 'purchase_year', 'purchase_month','trans_count_by_ym_merchant_card']
    trans_count_by_ym_merchant_card['trans_count_diff_by_ym_merchant_card'] = trans_count_by_ym_merchant_card.apply(lambda x : trans_count_diff_by_ym_merchant(x), axis=1)
    trans_count_by_ym_merchant_card['trans_count_by_ym_merchant_card_rate_as_ym'] = trans_count_by_ym_merchant_card.apply(lambda x : x['trans_count_by_ym_merchant_card']/trans_count_by_ym[x['purchase_year']][x['purchase_month']]*100, axis=1)
    trans_count_by_ym_merchant_card['trans_count_by_ym_merchant_card_rate_as_merchant'] = trans_count_by_ym_merchant_card.apply(lambda x : x['trans_count_by_ym_merchant_card']/trans_count_by_merchant[x['merchant_id']]*100, axis=1)
    trans_count_by_ym_merchant_card['trans_count_by_ym_merchant_card_rate_as_ym_merchant'] = trans_count_by_ym_merchant_card.apply(lambda x : x['trans_count_by_ym_merchant_card']/trans_count_by_ym_merchant[x['merchant_id']][x['purchase_year']][x['purchase_month']]*100, axis=1)
    
    aggs = {
        'trans_count_by_ym_merchant_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_by_ym_merchant_card_rate_as_ym':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_by_ym_merchant_card_rate_as_merchant' : ['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_by_ym_merchant_card_rate_as_ym_merchant' : ['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_diff_by_ym_merchant_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
    }
    return trans_count_by_ym_merchant_card.groupby('card_id').agg(aggs)

In [14]:
def purchase_amount_by_ym_city(df):
    purchase_amount_sum_by_ym = df.groupby(['purchase_year', 'purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_city = df.groupby('city_id')['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_ym_city = df.groupby(['city_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_ym_city_card = df.groupby(['card_id', 'city_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_ym_city_dict = {}
    for (card_id, city, year, month), val in zip(purchase_amount_sum_by_ym_city_card.index.tolist(), purchase_amount_sum_by_ym_city_card.values):
        if card_id not in purchase_amount_sum_by_ym_city_dict : purchase_amount_sum_by_ym_city_dict[card_id] = {}
        if city not in purchase_amount_sum_by_ym_city_dict[card_id] : purchase_amount_sum_by_ym_city_dict[card_id][city] = {}
        if year not in purchase_amount_sum_by_ym_city_dict[card_id][city]: purchase_amount_sum_by_ym_city_dict[card_id][city][year] = {}
        purchase_amount_sum_by_ym_city_dict[card_id][city][year][month] = val
    def purchase_amount_diff_by_ym_city(x):
        month = x['purchase_month']
        try :
            if month==1 : temp = purchase_amount_sum_by_ym_city_dict[x['card_id']][x['city_id']][x['purchase_year']-1][12]
            else : temp = purchase_amount_sum_by_ym_city_dict[x['card_id']][x['city_id']][x['purchase_year']][month-1]
        except KeyError : return np.nan
        return x['purchase_amount_sum_by_ym_city_card']-temp
    purchase_amount_sum_by_ym_city_card = purchase_amount_sum_by_ym_city_card.reset_index()
    purchase_amount_sum_by_ym_city_card.columns = ['card_id', 'city_id', 'purchase_year', 'purchase_month', 'purchase_amount_sum_by_ym_city_card']
    purchase_amount_sum_by_ym_city_card['purchase_amount_diff_by_ym_city_card'] = purchase_amount_sum_by_ym_city_card.apply(lambda x : purchase_amount_diff_by_ym_city(x), axis=1)
    purchase_amount_sum_by_ym_city_card['purchase_amount_sum_by_ym_city_card_rate_as_ym'] = purchase_amount_sum_by_ym_city_card.apply(lambda x : x['purchase_amount_sum_by_ym_city_card']/purchase_amount_sum_by_ym[x['purchase_year']][x['purchase_month']]*100, axis=1)
    purchase_amount_sum_by_ym_city_card['purchase_amount_sum_by_ym_city_card_rate_as_city'] = purchase_amount_sum_by_ym_city_card.apply(lambda x : x['purchase_amount_sum_by_ym_city_card']/purchase_amount_sum_by_city[x['city_id']]*100, axis=1)
    purchase_amount_sum_by_ym_city_card['purchase_amount_sum_by_ym_city_card_rate_as_ym_city'] = purchase_amount_sum_by_ym_city_card.apply(lambda x : x['purchase_amount_sum_by_ym_city_card']/purchase_amount_sum_by_ym_city[x['city_id']][x['purchase_year']][x['purchase_month']]*100, axis=1)
    
    aggs = {
        'purchase_amount_sum_by_ym_city_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_ym_city_card_rate_as_ym':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_ym_city_card_rate_as_city' : ['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_ym_city_card_rate_as_ym_city' : ['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_diff_by_ym_city_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
    }
    return purchase_amount_sum_by_ym_city_card.groupby('card_id').agg(aggs)

In [15]:
def trans_count_by_ym_city(df):
    trans_count_by_ym = df.groupby(['purchase_year', 'purchase_month'])['purchase_amount_new'].agg('count')
    trans_count_by_city = df.groupby('city_id')['purchase_amount_new'].agg('count')
    trans_count_by_ym_city = df.groupby(['city_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('count')
    trans_count_by_ym_city_card = df.groupby(['card_id', 'city_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].agg('count')
    trans_count_by_ym_city_dict = {}
    for (card_id, city, year, month), val in zip(trans_count_by_ym_city_card.index.tolist(), trans_count_by_ym_city_card.values):
        if card_id not in trans_count_by_ym_city_dict : trans_count_by_ym_city_dict[card_id] = {}
        if city not in trans_count_by_ym_city_dict[card_id] : trans_count_by_ym_city_dict[card_id][city] = {}
        if year not in trans_count_by_ym_city_dict[card_id][city]: trans_count_by_ym_city_dict[card_id][city][year] = {}
        trans_count_by_ym_city_dict[card_id][city][year][month] = val
    def trans_count_diff_by_ym_city(x):
        month = x['purchase_month']
        try :
            if month==1 : temp = trans_count_by_ym_city_dict[x['card_id']][x['city_id']][x['purchase_year']-1][12]
            else : temp = trans_count_by_ym_city_dict[x['card_id']][x['city_id']][x['purchase_year']][month-1]
        except KeyError : return np.nan
        return x['trans_count_by_ym_city_card']-temp
    trans_count_by_ym_city_card = trans_count_by_ym_city_card.reset_index()
    trans_count_by_ym_city_card.columns = ['card_id', 'city_id', 'purchase_year', 'purchase_month','trans_count_by_ym_city_card']
    trans_count_by_ym_city_card['trans_count_diff_by_ym_city_card'] = trans_count_by_ym_city_card.apply(lambda x : trans_count_diff_by_ym_city(x), axis=1)
    trans_count_by_ym_city_card['trans_count_by_ym_city_card_rate_as_ym'] = trans_count_by_ym_city_card.apply(lambda x : x['trans_count_by_ym_city_card']/trans_count_by_ym[x['purchase_year']][x['purchase_month']]*100, axis=1)
    trans_count_by_ym_city_card['trans_count_by_ym_city_card_rate_as_city'] = trans_count_by_ym_city_card.apply(lambda x : x['trans_count_by_ym_city_card']/trans_count_by_city[x['city_id']]*100, axis=1)
    trans_count_by_ym_city_card['trans_count_by_ym_city_card_rate_as_ym_city'] = trans_count_by_ym_city_card.apply(lambda x : x['trans_count_by_ym_city_card']/trans_count_by_ym_city[x['city_id']][x['purchase_year']][x['purchase_month']]*100, axis=1)
    
    aggs = {
        'trans_count_by_ym_city_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_by_ym_city_card_rate_as_ym':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_by_ym_city_card_rate_as_city' : ['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_by_ym_city_card_rate_as_ym_city' : ['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_diff_by_ym_city_card':['min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
    }
    return trans_count_by_ym_city_card.groupby('card_id').agg(aggs)

In [6]:
refDate = dt(2017,1,1,0,0,8)
daysOfMonth = [calendar.monthrange(2017, i)[1] for i in range(1,12)]
for i, v in enumerate(daysOfMonth[1:]):
    daysOfMonth[i+1] = v+daysOfMonth[i]
daysOfMonth.append(0)
def fromRefDate(x):
    ret = (x.year-refDate.year)*365
    ret += daysOfMonth[x.month-2]
    ret += x.day-refDate.day
    return ret
    
def transFE(df):
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['price'] = df['purchase_amount'] / df['installments']
    df['purchase_ymd'] = pd.DatetimeIndex(df['purchase_date']).date

    # purchase date
    df['purchase_year'] = df['purchase_date'].dt.year
    df['purchase_month'] = df['purchase_date'].dt.month
    df['purchase_day'] = df['purchase_date'].dt.day
    df['purchase_hour'] = df['purchase_date'].dt.hour
    df['purchase_dayofweek'] = df['purchase_date'].dt.dayofweek
    df['purchase_weekofyear'] = df['purchase_date'].dt.weekofyear
    df['purchase_weekend'] = (df['purchase_date'].dt.weekday >=5).astype(int)
    # anniversary
    df['Christmas_Day_2017']=(pd.to_datetime('2017-12-25')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    df['Mothers_Day_2017']=(pd.to_datetime('2017-06-04')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    df['fathers_day_2017']=(pd.to_datetime('2017-08-13')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    df['Children_day_2017']=(pd.to_datetime('2017-10-12')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    df['Valentine_Day_2017']=(pd.to_datetime('2017-06-12')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    df['Black_Friday_2017']=(pd.to_datetime('2017-11-24') - df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    df['Mothers_Day_2018']=(pd.to_datetime('2018-05-13')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #
    df['fromRefDate'] = df['purchase_date'].apply(fromRefDate)

    # 날짜 기준 : 마지막 거래일
    df['purchase_date_total_day'] = pd.to_timedelta(df['purchase_date']).dt.total_seconds() / (60 * 60 * 24)
    df['month_diff_from_trade'] = ((dt(2018, 4, 30, 23, 59, 59) - df['purchase_date']).dt.days)//30
    df['month_diff_from_trade'] += df['month_lag']
    df['month_diff_from_today'] = ((dt.today() - df['purchase_date']).dt.days)//30
    df['month_diff_from_today'] += df['month_lag']
    
    # cat
    for col in ['category_2','category_3']:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        df[col+'_min'] = df.groupby([col])['purchase_amount'].transform('min')
        df[col+'_max'] = df.groupby([col])['purchase_amount'].transform('max')
        df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum')
        
    # purchase
    df['purchase_amount_trim'] = df['purchase_amount'].apply(lambda x: min(x, 0.8))
    return df
    
def transAgg(df):
    aggs = {     
        # about time
        'fromRefDate':['sum','min','max','mean','median','var',quantileRange],
        'purchase_year':[mode, 'nunique', 'mean', 'max', 'min'],
        'purchase_month':[mode, 'nunique', 'mean', 'max', 'min'],
        'purchase_day':[mode, 'nunique', 'mean', 'var', 'max', 'min', 'skew'],
        'purchase_hour':[mode, 'nunique', 'mean', 'var', 'max', 'min', 'skew'],
        'purchase_date': ['max','min'],
        'month_lag':['sum','mean', 'median', 'var', 'max', 'min',quantileRange, 'skew'],
        'purchase_dayofweek': [mode, 'nunique', 'mean', 'max', 'min'],
        'purchase_weekofyear': [mode, 'nunique', 'mean', 'max', 'min'],
        'purchase_weekend': [mode, 'sum', 'mean'],
        'Christmas_Day_2017':['mean'],
        'Children_day_2017':['mean'],
        'Black_Friday_2017':['mean'],
        'Mothers_Day_2018':['mean'],
        'purchase_date_total_day':['max','min','mean','var','skew'],
        'month_diff_from_trade':['max','min','mean','var','skew'],
        'month_diff_from_today':['max','min','mean','var','skew'],
        
        # about location
        'city_id' : ['count', mode, 'nunique'],
        'state_id':[mode,'nunique'],
        'subsector_id':[mode, 'nunique'],
        
        # about merchant
        'merchant_category_id' : [mode,'nunique'],
        'merchant_id' : ['nunique'],
        
        # about transaction
        'installments' : ['sum','mean', 'median', 'var', 'max','min',quantileRange, null_cnt],
        'installments_new' : ['sum','mean', 'median', 'var', 'max','min',quantileRange, null_cnt],
        'purchase_amount':['sum','mean', 'median', 'var', 'max', 'min',quantileRange, 'skew', over_550],
        'purchase_amount_new':['sum','mean', 'median', 'var', 'max', 'min',quantileRange, 'skew', over_550],
        'purchase_amount_trim':['sum', 'mean', 'var', 'max', 'min', 'skew'],
        # about category
        'authorized_flag_new' : ['mean', 'sum'],
        'category_1_new':[mode, 'nunique', 'mean', 'var','sum',],
        'category_3_new' : [mode, 'nunique', 'mean', 'var',],
        'category_2_new' : [mode, 'mean', 'var','nunique'],
    }
    for col in ['category_2','category_3']:
            aggs[col+'_mean'] = ['mean']
            aggs[col+'_min'] = ['mean']
            aggs[col+'_max'] = ['mean']
            aggs[col+'_sum'] = ['mean']
    return df.groupby('card_id').agg(aggs).reset_index()

In [20]:
# new features
def transFE2(df):    
    '''
    --- year X month ---
    - purchase ym : year + month
        - (groupby : card id, agg : count) : 월별 거래횟수
    '''
    def ym_merge(x):
        year = str(x['purchase_year'])
        month = str(x['purchase_month'])
        if len(month)<2 : return year+'0'+month
        else : return year+month
    df['purchase_ym'] = df.apply(lambda x: ym_merge(x), axis=1)
    
    '''
    (card_id는 아무거나 찍은 거)
    1. city별 월별 거래횟수를 구해둔다 -> out
    2. 월별 거래횟수를 구해둔다 
    3. 해당 도시가 월별 거래횟수를 얼마나 차지하는지. 백분율을 구한다. -> out
    '''
    df['trans_count_by_city_ym'] = df.groupby(['city_id', 'purchase_year', 'purchase_month'])['card_id'].transform('count')
    trans_count_by_city_ym = df.groupby(['purchase_year','purchase_month'])['card_id'].agg('count')
    trans_count_by_city_ym_dict = {}
    for (year, month), val in zip(trans_count_by_city_ym.index.tolist(), trans_count_by_city_ym.values):
        if year not in trans_count_by_city_ym_dict : trans_count_by_city_ym_dict[year] = {}
        trans_count_by_city_ym_dict[year][month] = val
    def trans_count_by_city_ym_rate(x):
        year = x['purchase_year']
        month = x['purchase_month']
        return x['trans_count_by_city_ym']/trans_count_by_city_ym_dict[year][month]*100
    df['trans_count_by_city_ym_rate'] = df.apply(lambda x : trans_count_by_city_ym_rate(x), axis=1)
    
    '''
    1. city별 월별 거래대금 총합을 구해둔다 -> out
    2. 월별 거래대금 총합을 구해둔다 
    3. 해당 도시가 월별 거래대금 총합을 얼마나 차지하는지. 백분율을 구한다. -> out
    '''
    df['purchase_amount_sum_by_city_ym'] = df.groupby(['city_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].transform('sum')
    purchase_amount_sum_by_city_ym = df.groupby(['purchase_year','purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_city_ym_dict = {}
    for (year, month), val in zip(purchase_amount_sum_by_city_ym.index.tolist(), purchase_amount_sum_by_city_ym.values):
        if year not in purchase_amount_sum_by_city_ym_dict : purchase_amount_sum_by_city_ym_dict[year] = {}
        purchase_amount_sum_by_city_ym_dict[year][month] = val
    def purchase_amount_sum_by_city_ym_rate(x):
        year = x['purchase_year']
        month = x['purchase_month']
        return x['purchase_amount_sum_by_city_ym']/purchase_amount_sum_by_city_ym_dict[year][month]*100
    df['purchase_amount_sum_by_city_ym_rate'] = df.apply(lambda x : purchase_amount_sum_by_city_ym_rate(x), axis=1)
    
    
    '''
    (card_id는 아무거나 찍은 거)
    1. merchant별 월별 거래횟수를 구해둔다 -> out
    2. 월별 거래횟수를 구해둔다 
    3. 해당 도시가 월별 거래횟수를 얼마나 차지하는지. 백분율을 구한다. -> out
    '''
    df['trans_count_by_merchant_ym'] = df.groupby(['merchant_id', 'purchase_year', 'purchase_month'])['card_id'].transform('count')
    trans_count_by_merchant_ym = df.groupby(['purchase_year','purchase_month'])['card_id'].agg('count')
    trans_count_by_merchant_ym_dict = {}
    for (year, month), val in zip(trans_count_by_merchant_ym.index.tolist(), trans_count_by_merchant_ym.values):
        if year not in trans_count_by_merchant_ym_dict : trans_count_by_merchant_ym_dict[year] = {}
        trans_count_by_merchant_ym_dict[year][month] = val
    def trans_count_by_merchant_ym_rate(x):
        year = x['purchase_year']
        month = x['purchase_month']
        return x['trans_count_by_merchant_ym']/trans_count_by_merchant_ym_dict[year][month]*100
    df['trans_count_by_merchant_ym_rate'] = df.apply(lambda x : trans_count_by_merchant_ym_rate(x), axis=1)
    
    '''
    1. merchant별 월별 거래대금 총합을 구해둔다 -> out
    2. 월별 거래대금 총합을 구해둔다 
    3. 해당 도시가 월별 거래대금 총합을 얼마나 차지하는지. 백분율을 구한다. -> out
    '''
    df['purchase_amount_sum_by_merchant_ym'] = df.groupby(['merchant_id', 'purchase_year', 'purchase_month'])['purchase_amount_new'].transform('sum')
    purchase_amount_sum_by_merchant_ym = df.groupby(['purchase_year','purchase_month'])['purchase_amount_new'].agg('sum')
    purchase_amount_sum_by_merchant_ym_dict = {}
    for (year, month), val in zip(purchase_amount_sum_by_merchant_ym.index.tolist(), purchase_amount_sum_by_merchant_ym.values):
        if year not in purchase_amount_sum_by_merchant_ym_dict : purchase_amount_sum_by_merchant_ym_dict[year] = {}
        purchase_amount_sum_by_merchant_ym_dict[year][month] = val
    def purchase_amount_sum_by_merchant_ym_rate(x):
        year = x['purchase_year']
        month = x['purchase_month']
        return x['purchase_amount_sum_by_merchant_ym']/purchase_amount_sum_by_merchant_ym_dict[year][month]*100
    df['purchase_amount_sum_by_merchant_ym_rate'] = df.apply(lambda x : purchase_amount_sum_by_merchant_ym_rate(x), axis=1)
    
    aggs = {
        'purchase_ym':[mode, 'nunique',],
        'trans_count_by_city_ym':['sum','min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_by_city_ym_rate':['sum','min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_city_ym':['sum','min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_city_ym_rate':['sum','min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_by_merchant_ym':['sum','min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'trans_count_by_merchant_ym_rate':['sum','min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_merchant_ym':['sum','min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
        'purchase_amount_sum_by_merchant_ym_rate':['sum','min', 'max', 'median', 'mean', 'var', 'skew', quantileRange,],
    }
    return df.groupby('card_id').agg(aggs).reset_index()

In [14]:
def mervisit(df):
# Mixing Card id and Mer id
    merchant_visit = df.groupby(['card_id', 'merchant_id'])['purchase_ymd'].agg('count')
    return merchant_visit.groupby(['card_id']).agg(["sum", "mean", "min", "max",'nunique','size',mode]).reset_index()

def mertry(df):
    merchant_try = df.groupby(['card_id', 'merchant_id','purchase_ymd'])['purchase_ymd'].agg('count')
    return merchant_try.groupby(['card_id']).agg([ "mean", "std", "min", "max",'nunique','size',mode]).reset_index()

In [5]:
def transYM(df):
    def applying(x):
        year = x['purchase_year']
        month = x['purchase_month']
        return (year-2017)*12+month-1
    df['purchase_ym_int'] = df.apply(applying, axis=1)
    return df


def transYMAgg(df):
    aggs = {
        'city_id':[modeKey, ],
       'merchant_category_id': [modeKey],
        'month_lag': [modeKey, top3_key_mean],
       'state_id': [modeKey],
       'subsector_id': [modeKey],
        'category_1_new': [modeKey],
       'category_3_new': [modeKey],
        'category_2_new': [modeKey],
        'installments': [modeKey, top3_key_mean],
        'purchase_ym_int': [modeKey],
    }
    return df.groupby('card_id').agg(aggs).reset_index()

In [11]:
def merIdCnvrt(df):
    merIdCnvrter = {}
    sortedMer = sorted(mer[['merchant_id', 'city_id', 'merchant_category_id', 'merchant_group_id']].values, key = operator.itemgetter(1,2,3,0))
    for i, (merId, *_) in enumerate(sortedMer):
        merIdCnvrter[merId] = i
    def merIdCnvrting(x):
        try: return merIdCnvrter[x]
        except KeyError : return np.nan
    df['merchant_id_new'] = df['merchant_id'].apply(merIdCnvrting)
    aggs = {
        'merchant_id_new' : [modeKey, top3_key_mean, 'sum', 'min', 'max', 'median', 'mean', 'var', quantileRange,],
    }
    return df.groupby('card_id').agg(aggs).reset_index()